In [1]:
import pandas as pd
import os
import csv

In [2]:
def FilesPath(path):
    filespath = []
    for root, dir, files in os.walk(path):
        for file in files:
            filespath.append(os.path.join(root, file))
    return filespath

Total_file = FilesPath('./suicide_data')
len(Total_file)

24

In [3]:
file_paths = []
merged_df = None

for file_path in (Total_file[1:len(Total_file)]):
    file_paths.append(file_path)

for file in file_paths:
    # 讀取檔案並跳過前兩行標題
    year = file.split('/')[-1].split('.')[0]  # 從檔名中提取年份
    df = pd.read_csv(file, encoding='utf-8', skiprows=2)
    
    # 重命名 'Total' 欄位，區分不同年份
    df.rename(columns={'Total': f'{year} Suicide Rate'}, inplace=True)
    
    # 合併檔案 (若 merged_df 是 None，則初始化為第一個 DataFrame)
    if merged_df is None:
        merged_df = df
    else:
        merged_df = pd.merge(merged_df, df, on='Category', how='inner')

In [4]:
rows = []
for row in merged_df:
    rows.append(row)
avg_df = merged_df
rate = avg_df[rows].select_dtypes(include='number')
avg_df['Average suicide rate'] = rate.mean(axis = 1)
avg_df.to_csv('merged_suicide_rates_years_1990_2019.csv', index=False)

In [5]:
path = './suicide-rate-by-country-2024.csv'
df = pd.read_csv(path, encoding = 'utf-8')
melted_df = pd.melt(df, 
                    id_vars=['Category'],  
                    var_name='SuicideRate_Type_Year',
                    value_name='RatePer100k')

melted_df[['SuicideRate_Type', 'Year']] = melted_df['SuicideRate_Type_Year'].str.extract(r'(.*)_RatePer100k_(\d{4})')
both_sexes_df = melted_df[melted_df['SuicideRate_Type'] == 'SuicideRate_BothSexes']
both_sexes_df.drop(columns=['SuicideRate_Type_Year', 'SuicideRate_Type'], inplace=True)
both_sexes_df = both_sexes_df.dropna()
both_sexes_df = both_sexes_df.pivot(index='Category', columns='Year', values='RatePer100k')

final_merged_df = pd.merge(merged_df, both_sexes_df, how='left', left_on='Category', right_on='Category')

rows = []
for row in final_merged_df:
    rows.append(row)
rate = final_merged_df[rows].select_dtypes(include='number')
final_merged_df['Average suicide rate'] = rate.mean(axis = 1)

final_merged_df.to_csv('final_merged_bothsexes_suicide_rates_1990_2021.csv', index=False)


/var/folders/c2/snb3yrcs1w38z7qd3r208vch0000gn/T/ipykernel_27337/2036512285.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  both_sexes_df.drop(columns=['SuicideRate_Type_Year', 'SuicideRate_Type'], inplace=True)


In [6]:
path = './suicide-rate-by-country-2024.csv'
df = pd.read_csv(path, encoding = 'utf-8')
melted_df = pd.melt(df, 
                    id_vars=['Category'],  
                    var_name='SuicideRate_Variable',
                    value_name='RatePer100k')

melted_df[['SuicideRate_Type', 'Year']] = melted_df['SuicideRate_Variable'].str.extract(r'SuicideRate_(.*)_RatePer100k_(\d{4})')

melted_df.rename(columns={'RatePer100k': 'Suicide Rate'}, inplace=True)
melted_df.drop(columns=['SuicideRate_Variable'], inplace=True)
melted_df = melted_df.dropna()

df = melted_df.pivot(index='Category', columns=['SuicideRate_Type', 'Year'], values='Suicide Rate')
df['Male_Avg'] = df[['Male']].mean(axis=1)
df['Female_Avg'] = df[['Female']].mean(axis=1)
df['Both_Avg'] = df[['BothSexes']].mean(axis=1)
df.to_csv('all_sexes_average_suicide_rates_years_2019_2021.csv', index=True)